# 6章 物流の最適ルートをコンサルティングする１０本ノック

ここでは、「物流」の基礎となる「輸送最適化」を検討するにあたっての基礎的な技術を習得します。  
実際の物流データからネットワーク構造を可視化する方法について学び、最適な物流計画を立案する流れを学んでいきます。

### ノック５１：物流に関するデータを読み込んでみよう

In [22]:
import pandas as pd
factories = pd.read_csv('tbl_factory.csv')
factories

,FCID,FCName,FCDemand,FCRegion
0,FC00001,東京工場,28,関東
1,FC00002,木更津工場,29,関東
2,FC00003,多摩工場,31,関東
3,FC00004,横須賀工場,25,関東
4,FC00005,仙台工場,21,東北
5,FC00006,山形工場,30,東北
6,FC00007,那須工場,25,東北
7,FC00008,青森工場,16,東北


In [23]:
warehouses = pd.read_csv('tbl_warehouse.csv')
warehouses

,WHID,WHName,WHSupply,WHRegion
0,WH00001,杉並倉庫,35,関東
1,WH00002,品川倉庫,41,関東
2,WH00003,豊洲倉庫,42,関東
3,WH00004,郡山倉庫,60,東北
4,WH00005,仙台倉庫,72,東北
5,WH00006,山形倉庫,65,東北


In [24]:
cost = pd.read_csv('rel_cost.csv')
cost

,RCostID,FCID,WHID,Cost
0,1,FC00001,WH00001,0.4
1,2,FC00001,WH00002,0.8
2,3,FC00001,WH00003,1.5
3,4,FC00002,WH00001,0.9
4,5,FC00002,WH00002,0.8
5,6,FC00002,WH00003,1.6
6,7,FC00003,WH00001,1.2
7,8,FC00003,WH00002,1.3
8,9,FC00003,WH00003,1.5
9,10,FC00004,WH00001,0.8


In [25]:
trans = pd.read_csv('tbl_transaction.csv')
trans

,TRID,TransactionDate,ToFC,FromWH,Quantity
0,0,2019-01-01 02:11:10,FC00004,WH00003,33
1,1,2019-01-01 06:12:42,FC00007,WH00006,19
2,2,2019-01-01 06:32:32,FC00006,WH00004,31
3,3,2019-01-01 07:17:06,FC00002,WH00003,18
4,4,2019-01-01 07:52:18,FC00001,WH00002,30
...,...,...,...,...,...
3995,3995,2019-12-31 11:27:51,FC00007,WH00006,29
3996,3996,2019-12-31 17:10:21,FC00002,WH00003,35
3997,3997,2019-12-31 19:35:07,FC00003,WH00001,36
3998,3998,2019-12-31 21:45:07,FC00005,WH00005,34


In [26]:
join_data = pd.merge(trans, cost, left_on=['ToFC', 'FromWH'], right_on=['FCID', 'WHID'], how='left')
join_data

,TRID,TransactionDate,ToFC,FromWH,Quantity,RCostID,FCID,WHID,Cost
0,0,2019-01-01 02:11:10,FC00004,WH00003,33,12,FC00004,WH00003,1.1
1,1,2019-01-01 06:12:42,FC00007,WH00006,19,21,FC00007,WH00006,1.3
2,2,2019-01-01 06:32:32,FC00006,WH00004,31,16,FC00006,WH00004,0.9
3,3,2019-01-01 07:17:06,FC00002,WH00003,18,6,FC00002,WH00003,1.6
4,4,2019-01-01 07:52:18,FC00001,WH00002,30,2,FC00001,WH00002,0.8
...,...,...,...,...,...,...,...,...,...
3995,3995,2019-12-31 11:27:51,FC00007,WH00006,29,21,FC00007,WH00006,1.3
3996,3996,2019-12-31 17:10:21,FC00002,WH00003,35,6,FC00002,WH00003,1.6
3997,3997,2019-12-31 19:35:07,FC00003,WH00001,36,7,FC00003,WH00001,1.2
3998,3998,2019-12-31 21:45:07,FC00005,WH00005,34,14,FC00005,WH00005,0.3


In [27]:
join_data = pd.merge(join_data, factories, on='FCID', how='left')
join_data

,TRID,TransactionDate,ToFC,FromWH,Quantity,RCostID,FCID,WHID,Cost,FCName,FCDemand,FCRegion
0,0,2019-01-01 02:11:10,FC00004,WH00003,33,12,FC00004,WH00003,1.1,横須賀工場,25,関東
1,1,2019-01-01 06:12:42,FC00007,WH00006,19,21,FC00007,WH00006,1.3,那須工場,25,東北
2,2,2019-01-01 06:32:32,FC00006,WH00004,31,16,FC00006,WH00004,0.9,山形工場,30,東北
3,3,2019-01-01 07:17:06,FC00002,WH00003,18,6,FC00002,WH00003,1.6,木更津工場,29,関東
4,4,2019-01-01 07:52:18,FC00001,WH00002,30,2,FC00001,WH00002,0.8,東京工場,28,関東
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3995,2019-12-31 11:27:51,FC00007,WH00006,29,21,FC00007,WH00006,1.3,那須工場,25,東北
3996,3996,2019-12-31 17:10:21,FC00002,WH00003,35,6,FC00002,WH00003,1.6,木更津工場,29,関東
3997,3997,2019-12-31 19:35:07,FC00003,WH00001,36,7,FC00003,WH00001,1.2,多摩工場,31,関東
3998,3998,2019-12-31 21:45:07,FC00005,WH00005,34,14,FC00005,WH00005,0.3,仙台工場,21,東北


In [28]:
join_data = pd.merge(join_data, warehouses, on='WHID', how='left')
join_data = join_data[['TransactionDate', 'Quantity', 'Cost', 'ToFC', 'FCName', 'FCDemand', 'FromWH', 'WHName', 'WHSupply', 'WHRegion']]
join_data

,TransactionDate,Quantity,Cost,ToFC,FCName,FCDemand,FromWH,WHName,WHSupply,WHRegion
0,2019-01-01 02:11:10,33,1.1,FC00004,横須賀工場,25,WH00003,豊洲倉庫,42,関東
1,2019-01-01 06:12:42,19,1.3,FC00007,那須工場,25,WH00006,山形倉庫,65,東北
2,2019-01-01 06:32:32,31,0.9,FC00006,山形工場,30,WH00004,郡山倉庫,60,東北
3,2019-01-01 07:17:06,18,1.6,FC00002,木更津工場,29,WH00003,豊洲倉庫,42,関東
4,2019-01-01 07:52:18,30,0.8,FC00001,東京工場,28,WH00002,品川倉庫,41,関東
...,...,...,...,...,...,...,...,...,...,...
3995,2019-12-31 11:27:51,29,1.3,FC00007,那須工場,25,WH00006,山形倉庫,65,東北
3996,2019-12-31 17:10:21,35,1.6,FC00002,木更津工場,29,WH00003,豊洲倉庫,42,関東
3997,2019-12-31 19:35:07,36,1.2,FC00003,多摩工場,31,WH00001,杉並倉庫,35,関東
3998,2019-12-31 21:45:07,34,0.3,FC00005,仙台工場,21,WH00005,仙台倉庫,72,東北


In [29]:
kanto = join_data.loc[join_data['WHRegion'] ==  '関東']
kanto

,TransactionDate,Quantity,Cost,ToFC,FCName,FCDemand,FromWH,WHName,WHSupply,WHRegion
0,2019-01-01 02:11:10,33,1.1,FC00004,横須賀工場,25,WH00003,豊洲倉庫,42,関東
3,2019-01-01 07:17:06,18,1.6,FC00002,木更津工場,29,WH00003,豊洲倉庫,42,関東
4,2019-01-01 07:52:18,30,0.8,FC00001,東京工場,28,WH00002,品川倉庫,41,関東
7,2019-01-01 09:09:30,12,1.5,FC00001,東京工場,28,WH00003,豊洲倉庫,42,関東
8,2019-01-01 10:52:55,27,1.5,FC00003,多摩工場,31,WH00003,豊洲倉庫,42,関東
...,...,...,...,...,...,...,...,...,...,...
3984,2019-12-30 12:29:42,22,0.9,FC00002,木更津工場,29,WH00001,杉並倉庫,35,関東
3987,2019-12-30 17:51:24,32,1.3,FC00003,多摩工場,31,WH00002,品川倉庫,41,関東
3992,2019-12-31 01:50:16,33,0.8,FC00001,東京工場,28,WH00002,品川倉庫,41,関東
3996,2019-12-31 17:10:21,35,1.6,FC00002,木更津工場,29,WH00003,豊洲倉庫,42,関東


In [30]:
tohoku = join_data.loc[join_data['WHRegion'] == '東北']
tohoku

,TransactionDate,Quantity,Cost,ToFC,FCName,FCDemand,FromWH,WHName,WHSupply,WHRegion
1,2019-01-01 06:12:42,19,1.3,FC00007,那須工場,25,WH00006,山形倉庫,65,東北
2,2019-01-01 06:32:32,31,0.9,FC00006,山形工場,30,WH00004,郡山倉庫,60,東北
5,2019-01-01 08:56:09,31,0.3,FC00005,仙台工場,21,WH00005,仙台倉庫,72,東北
6,2019-01-01 09:00:15,33,0.7,FC00006,山形工場,30,WH00006,山形倉庫,65,東北
9,2019-01-01 14:12:51,21,0.7,FC00006,山形工場,30,WH00006,山形倉庫,65,東北
...,...,...,...,...,...,...,...,...,...,...
3993,2019-12-31 03:35:04,20,0.7,FC00006,山形工場,30,WH00006,山形倉庫,65,東北
3994,2019-12-31 07:05:56,11,0.8,FC00005,仙台工場,21,WH00004,郡山倉庫,60,東北
3995,2019-12-31 11:27:51,29,1.3,FC00007,那須工場,25,WH00006,山形倉庫,65,東北
3998,2019-12-31 21:45:07,34,0.3,FC00005,仙台工場,21,WH00005,仙台倉庫,72,東北


### ノック５２：現状の輸送量、コストを確認してみよう

In [37]:
print(f'関東支社の総コスト: {kanto["Cost"].sum()}万円')
print(f'東北支社の総コスト: {tohoku["Cost"].sum()}万円')

関東支社の総コスト: 2189.3万円
東北支社の総コスト: 2062.0万円


In [42]:
print(f'関東支社の部品一つ当たりの輸送コスト: {int((kanto["Cost"].sum() / kanto["Quantity"].sum()) * 10000)}円')
print(f'東北支社の部品一つ当たりの輸送コスト: {int((tohoku["Cost"].sum() / tohoku["Quantity"].sum()) * 10000)}円')

関東支社の部品一つ当たりの輸送コスト: 445円
東北支社の部品一つ当たりの輸送コスト: 410円


In [46]:
cost_chk = pd.merge(cost, factories, on='FCID', how='left')
print(f'関東支社の平均輸送コスト: {cost_chk.loc[cost_chk["FCRegion"] == "関東"]["Cost"].mean()}万円')
print(f'東北支社の平均輸送コスト: {cost_chk.loc[cost_chk["FCRegion"] == "東北"]["Cost"].mean()}万円')

関東支社の平均輸送コスト: 1.075万円
東北支社の平均輸送コスト: 1.05万円


### ノック５３：ネットワークを可視化してみよう

### ノック５４：ネットワークにノードを追加してみよう

### ノック５５：ルートの重みづけを実施しよう

### ノック５６：輸送ルート情報を読み込んでみよう

### ノック５７：輸送ルート情報からネットワークを可視化してみよう

### ノック５８：輸送コスト関数を作成しよう

### ノック５９：制約条件を作ってみよう

### ノック６０：輸送ルートを変更して、輸送コスト関数の変化を確認しよう